In [1]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('Fake_News.csv', encoding='ISO-8859-1', usecols=['title', 'text', 'label'])
df.head()


C:\Users\huzef\AppData\Local\Temp\ipykernel_6136\2045608207.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Fake_News.csv', encoding='ISO-8859-1', usecols=['title', 'text', 'label'])


,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,NaN,Did they post their votes for Hillary already?,1
2,UNBELIEVABLE! OBAMAâS ATTORNEY GENERAL SAYS ...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [8]:
df.isnull().sum()

title    2961
text     3609
label    4321
dtype: int64

In [9]:
df.shape

(78097, 3)

In [14]:
df.dropna(inplace=True)

In [15]:
df.shape

(73177, 3)

In [16]:
x=df.drop('label',axis=1)

x.shape

(73177, 2)

In [17]:
y=df['label']
y.shape

(73177,)

In [18]:
import tensorflow as tf

In [147]:
from tensorflow.keras.layers import Embedding ,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM , Bidirectional
from tensorflow.keras.layers import Dense

In [148]:
voc_size=70000

In [149]:
msg=x.copy()

In [150]:
msg['title'][0]

'LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO]'

In [151]:
msg

,title,text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...
2,UNBELIEVABLE! OBAMAâS ATTORNEY GENERAL SAYS ...,"Now, most of the demonstrators gathered last ..."
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will..."
5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...
...,...,...
78092,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...
78093,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n..."
78094,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...
78095,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trumpâs comba...


In [152]:


#msg.reset_index(inplace=True)

In [153]:
import nltk
import re
from nltk.corpus import stopwords

DATA PREPROCESSING

In [154]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(msg)):
    rev=re.sub('[^a-zA-Z]',' ',msg['title'][i])
    rev=rev.lower()
    rev=rev.split()
    rev=[ps.stem(word)for word in rev if not word in stopwords.words('english')]
    rev=' '.join(rev)
    corpus.append(rev)
corpus

KeyboardInterrupt: 

In [ ]:
corpus[1]

'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video'

In [ ]:
y.dtype

dtype('O')

One Hot Representation

In [ ]:
onehot_repres=[one_hot(words,voc_size)for words in corpus]
onehot_repres[0]

[22148,
 63990,
 54897,
 25031,
 26282,
 43419,
 55584,
 49319,
 65167,
 30594,
 17489,
 23661]

Padding : MAke all sentences of equal len

In [ ]:
sent_len=20
paddded_docs=pad_sequences(onehot_repres,padding='pre',maxlen=sent_len)

In [ ]:
paddded_docs[0]

array([    0,     0,     0,     0,     0,     0,     0,     0, 22148,
       63990, 54897, 25031, 26282, 43419, 55584, 49319, 65167, 30594,
       17489, 23661])

Creating a model


In [ ]:
vector_feature=40  
model=Sequential()
model.add(Embedding(voc_size,vector_feature,input_length=sent_len))
model.add(Dropout(0.3))
#model.add(LSTM(100))
#for bidirectional
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

c:\Users\huzef\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
len(paddded_docs),y.shape

(73177, (73177,))

In [ ]:
y.dtype

dtype('O')

In [ ]:
x_final=np.array(paddded_docs)
#y_final=np.array(y)` is converting the list `y` into a NumPy array and assigning it to the variable `y_final`. This can be useful for further processing or analysis that requires NumPy arrays instead of lists.
y_final=np.array(y)

In [ ]:
#This code snippet is performing label encoding on a list of labels `y`.
# Convert everything to string first
#y contains a mix of string float  and int to converting it to int
y_cleaned = [str(label).strip().lower() for label in y]

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y_cleaned)
y_final = np.array(y_encoded)



In [ ]:
y_final.dtype

dtype('int64')

In [ ]:
y_final.dtype

dtype('int64')

In [ ]:
x_final.shape,y_final.shape

((73177, 20), (73177,))

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.2,random_state=42)

In [ ]:
y_train.dtype

dtype('int64')

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
915/915 ━━━━━━━━━━━━━━━━━━━━ 25s 25ms/step - accuracy: 0.0000e+00 - loss: -851.5170 - val_accuracy: 6.8325e-05 - val_loss: -2466.2891
Epoch 2/10
915/915 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - accuracy: 0.0000e+00 - loss: -2993.6736 - val_accuracy: 6.8325e-05 - val_loss: -4455.9814
Epoch 3/10
915/915 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - accuracy: 0.0000e+00 - loss: -5113.0142 - val_accuracy: 6.8325e-05 - val_loss: -6442.7642
Epoch 4/10
915/915 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - accuracy: 0.0000e+00 - loss: -7053.5376 - val_accuracy: 6.8325e-05 - val_loss: -8442.0449
Epoch 5/10
915/915 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - accuracy: 0.0000e+00 - loss: -9039.2822 - val_accuracy: 6.8325e-05 - val_loss: -10445.4922
Epoch 6/10
915/915 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - accuracy: 0.0000e+00 - loss: -11562.2686 - val_accuracy: 6.8325e-05 - val_loss: -12470.9746
Epoch 7/10
915/915 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - accuracy: 0.0000e+00 - loss: -13097.7842 - val_accuracy: 6.8325e-0

Performanece metrics

In [ ]:
y_pred=(model.predict(x_test))

458/458 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [ ]:
y_pred=np.where(y_pred>0.6,1,0)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [ ]:
confusion_matrix(y_test,y_pred)

array([[   0,    1,    0, ...,    0,    0,    0],
       [   0,    1,    0, ...,    0,    0,    0],
       [   0, 6835,    0, ...,    0,    0,    0],
       ...,
       [   0,    1,    0, ...,    0,    0,    0],
       [   0,    1,    0, ...,    0,    0,    0],
       [   0,    1,    0, ...,    0,    0,    0]], dtype=int64)

In [ ]:
accuracy_score(y_test,y_pred)


6.832467887400929e-05

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      1.00      0.00         1
           4       0.00      0.00      0.00      6835
           5       0.00      0.00      0.00       200
          10       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         2
          20       0.00      0.00      0.00         1
          21       0.00      0.00      0.00      7051
          22       0.00      0.00      0.00       198
          24       0.00      0.00      0.00         1
          26       0.00      0.00      0.00         1
          27       0.00      0.00      0.00         1
          29       0.00      0.00      0.00         1
          32       0.00      0.00      0.00         1
          34       0.00      0.00      0.00         1
          38       0.00      0.00      0.00         1
          51       0.00      0.00      0.00         2
          55       0.00    

c:\Users\huzef\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\huzef\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\huzef\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
